In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Ir_example').getOrCreate()

In [5]:
 from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.show() #see how the data is

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.4972677251
12.6556511492
39.5776680195
4.08262063295
587.951053968


In [10]:
from pyspark.ml.linalg import Vectors

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website', 'Length of Membership','Yearly Amount Spent'],outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
output.head(1)

[Row(Email=u'mstephenson@fernandez.com', Address=u'835 Frank TunnelWrightmouth, MI 82180-9605', Avatar=u'Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [16]:
final_d = output.select('features','Yearly Amount Spent')

In [17]:
final_d.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data, test_data = final_d.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean| 504.22381563457907|
| stddev|    78.719748694865|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean| 488.58758195308747|
| stddev|  79.80300239727542|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [21]:
linear_regression = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
linear_regression_model = linear_regression.fit(train_data)

In [24]:
test_result = linear_regression_model.evaluate(test_data)

In [25]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|                 0.0|
|-2.21689333557151...|
|-1.36424205265939...|
|2.671640686457976...|
|4.320099833421409...|
|-6.25277607468888...|
|-5.68434188608080...|
|1.648459146963432...|
|2.955857780762016...|
|-3.58113538823090...|
|-1.36424205265939...|
|5.400124791776761...|
|-1.53477230924181...|
|-1.76214598468504...|
|-2.44426701101474...|
|2.842170943040400...|
|3.637978807091713...|
|1.762145984685048...|
|-3.46744855050928...|
|-9.09494701772928...|
+--------------------+
only showing top 20 rows



In [26]:
test_result.rootMeanSquaredError

2.2270911126313765e-12

In [27]:
test_result.r2

1.0

In [29]:
unlabeled_data = test_data.select('features')

In [30]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.3091926408918...|
|[31.3895854806643...|
|[31.4474464941278...|
|[31.5741380228732...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.9262720263601...|
|[31.9365486184489...|
|[31.9563005605233...|
|[32.0085045178551...|
|[32.0180740106320...|
|[32.0215955013870...|
+--------------------+
only showing top 20 rows



In [31]:
pred = linear_regression_model.transform(unlabeled_data)

In [32]:
pred.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|442.06441375806565|
|[30.7377203726281...| 461.7807421962321|
|[30.9716756438877...| 494.6386097568941|
|[31.0662181616375...| 448.9332932076717|
|[31.1239743499119...|486.94705383976145|
|[31.3091926408918...|432.72071783993425|
|[31.3895854806643...|410.06961105998295|
|[31.4474464941278...|418.60274209522237|
|[31.5741380228732...| 544.4092721605839|
|[31.6098395733896...|444.54554965111174|
|[31.6548096756927...| 475.2634237275499|
|[31.6739155032749...| 475.7250679098758|
|[31.8209982016720...| 424.6752810132149|
|[31.8512531286083...|472.99224666680016|
|[31.9262720263601...|392.20493344432884|
|[31.9365486184489...|427.19938489532535|
|[31.9563005605233...| 547.1259317471952|
|[32.0085045178551...|443.19722102875363|
|[32.0180740106320...|357.78311074531877|
|[32.0215955013870...| 521.5721747578283|
+--------------------+------------